# DIY Spotify Wrapped 2022

# 1. Import necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import requests

import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials 
from tqdm import tqdm_notebook
import requests

import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from datetime import timedelta
import calendar

# 2. Connect to Spotify API

In [ ]:
# Generate token
username = '{insert_your_username}'
client_id = '{insert_client_id}'
client_secret = '{insert_client_secret}'

client_credentials = SpotifyClientCredentials(client_id, client_secret)

sp = spotipy.Spotify(client_credentials_manager = client_credentials)

# 3. Data pre-processing

## 3.1. Load raw data

In [ ]:
# Write a function to convert json files of streaming histories into a single dataframe
def get_streaming_hist_raw(json_file_list):
    result_df = pd.DataFrame()
    for json_file in json_file_list:
        # Open json file
        with open(json_file) as streaming_history_json:
            streaming_history_string = json.load(streaming_history_json)
        
        # Convert json file into dataframe
        streaming_history_df = pd.json_normalize(streaming_history_string)
        
        # Append to the combined dataframe
        result_df = pd.concat([result_df, streaming_history_df])
        
    # After all json have been added to the dataframe, drop duplicates
    result_df = result_df.drop_duplicates()
    result_df = result_df.rename(columns={'master_metadata_track_name':'trackName', 'master_metadata_album_artist_name':'artistName', 'master_metadata_album_album_name':'albumName'})
    
    return result_df

In [ ]:
# Apply the function to get streaming history dataframe
json_file_list = ['endsong_0.json', 'endsong_1.json', 'endsong_2.json', 'endsong_3.json', 'endsong_4.json', 'endsong_5.json', 'endsong_6.json', 'endsong_7.json', 'endsong_8.json', 'endsong_9.json']
streaming_hist_df = get_streaming_hist_raw(json_file_list)

In [ ]:
streaming_hist_df.info()

##  3.2. Get track info

In [ ]:
# Write a function to create a dictionary of trackNames and artistNames
# This will be an input for later
def get_trackName_artistName_dict(df):
    # Drop duplicates
    unique_artistName_trackName_df = df[['artistName','trackName']].drop_duplicates()
    
    # Convert df into lists
    trackName_list = unique_artistName_trackName_df['trackName'].to_list()
    artistName_list = unique_artistName_trackName_df['artistName'].to_list()
    
    # Create dictionary
    trackName_artistName_dict = {}
    for trackName in trackName_list:
        for artistName in artistName_list:
            trackName_artistName_dict[trackName] = artistName
            artistName_list.remove(artistName)
            break
    return trackName_artistName_dict

In [ ]:
# Write a function to get track info
def get_track_info(dict):
    # Create empty list
    track_id_list = []
    track_name_list = []
    track_popularity_list = []
    track_duration_list = []
    track_is_local_list = []
    artist_id_list = []
    artist_name_list = []
    album_id_list = []
    album_release_date_list = []
    
    # Declare counter
    count = 1
    
    # Retrieve track info
    for trackName, artistName in dict.items():        
        try:
            json = sp.search(q=f'{trackName} artist: {artistName}', type='track')
            track_info_items = json['tracks']['items'][0]
            print(f'track {count}: {trackName} - {artistName}... SUCCESSFUL')
        except Exception:
            try:
                json = sp.search(q=f'{trackName}', type='track')
                track_info_items = json['tracks']['items'][0]
                print(f'track {count}: {trackName} - {artistName}... SUCCESSFUL')
            except Exception:
                print(f'track {count}: {trackName} - {artistName}... FAILED')
                pass
        
        # Extract info to separate lists
        try:
            track_id_list.append(track_info_items['id']) # track_id
        except Exception:
            track_id_list.append('Unidentified')
        
        try:
            track_popularity_list.append(track_info_items['popularity']) # trackPopularity
        except Exception:
            track_popularity_list.append('Unidentified')
            
        try:
            track_duration_list.append(track_info_items['duration_ms']) # trackDurationMs
        except Exception:
            track_duration_list.append('Unidentified')
        
        try:
            track_is_local_list.append(track_info_items['is_local']) #trackIsLocal
        except Exception:
            track_is_local_list.append('Unidentified')
            
        try:
            artist_id_list.append(track_info_items['artists'][0]['id']) # artistId
        except Exception:
            artist_id_list.append('Unidentified')
            
        try:
            album_id_list.append(track_info_items['album']['id']) #album_id
        except Exception:
            album_id_list.append('Unidentified')
            
        try:
            album_release_date_list.append(track_info_items['album']['release_date']) #album_release_date
        except Exception:
            album_release_date_list.append('Unidentified')
            
        count+=1
        
    
    # Transform lists into a single dataframe
    track_name_list = list(dict.keys())
    artist_name_list = list(dict.values())
    data_tuples = list(zip(track_id_list, track_name_list, track_popularity_list, track_duration_list, track_is_local_list, artist_id_list, artist_name_list, album_id_list, album_release_date_list))
    col_names = ['trackId','trackName','trackPopularity','trackDurationMs','trackIsLocal','artistId','artistName','albumId','albumReleaseDate']
    df = pd.DataFrame(data_tuples, columns=col_names)
    df = df.drop_duplicates().reset_index(drop=True)
    
    return df

In [ ]:
%%time
# Apply the function to create a dictionary of trackNames and artistNames
trackName_artistName_dict = get_trackName_artistName_dict(streaming_hist_df)

# Apply the function to get track info
track_reference_df = get_track_info(trackName_artistName_dict)

##  3.3. Get track's audio features

In [ ]:
# Write a function to get audio features of a single track
def get_audio_features(track_id: str) -> dict:
    try:
        features = sp.audio_features(track_id)
        return features[0]
    except:
        return None

In [ ]:
# Write a function to populate the audio features of a trackId list
def get_audio_features_df(track_id_list: list) -> dict:
    # Create empty dictionary
    audio_features_dict = {}
    
    # Declare counter
    count = 1
    
    # Retrieve audio features of a trackId list
    for trackId in track_id_list:
        audio_features = get_audio_features(trackId)
        if audio_features:
            audio_features_dict[trackId] = audio_features # Create a dictionary with trackId as key and audio_features as value
            print(f'track {count}: {trackId}...SUCCESSFUL')
        else:
            print(f'track {count}: {trackId}...FAILED')
        count += 1
            
    # Transform dictionary into a list
    audio_feature_list = []
    for trackId, audio_features in audio_features_dict.items():
        audio_feature_list.append({'trackId': trackId, **audio_features})
        
    # Transfrom list into a dataframe
    audio_feature_df = pd.DataFrame(audio_feature_list)
    audio_feature_df = audio_feature_df.drop(columns={'id','uri','track_href','analysis_url'})
    
    return audio_feature_df

In [ ]:
%%time

# Apply the function to get the audio features
track_id_list = track_reference_df['trackId'].unique().tolist()
audio_feature_df = get_audio_features_df(track_id_list)

## 3.4. Get artist genre and popularity

In [ ]:
# Write a function to get artist info
def get_artist_info_df(artist_id_list: list) -> dict:
    # Create empty lists
    artist_genre_list = []
    artist_popularity_list = []
    artist_follower_count_list = []
    
    # Declare a counter
    count = 1
    
    # Retrieve artist info - Search by artistId
    for artistId in artist_id_list:
        try:
            json = sp.artist(artistId)
            print(f'artist {count}: {artistId}... SUCCESSFUL')
        except Exception:
            print(f'artist {count}: {artistId}... FAILED')
            pass
            
        try:
            artist_genre_list.append(json['genres'][0]) # artist_genre, select the first entry
        except Exception:
            artist_genre_list.append('Unidentified')
            
        try:
            artist_popularity_list.append(json['popularity']) # artist_popularity
        except Exception:
            artist_popularity_list.append('Unidentified')
            
        try:
            artist_follower_count_list.append(json['followers']['total']) # artist_follower_count
        except Exception:
            artist_follower_count_list.append('Unidentified')
            
        count+=1
        
    
    # Transform lists into a single dataframe
    data_tuples = list(zip(artist_id_list, artist_genre_list, artist_popularity_list, artist_follower_count_list))
    col_names = ['artistId','artistGenre', 'artistPopularity', 'artistFollowerCount']
    df = pd.DataFrame(data_tuples, columns=col_names)
    df = df.drop_duplicates().reset_index(drop=True)    
    
    return df

In [ ]:
%%time

# Apply the function to get the artist info
artist_id_list = track_reference_df['artistId'].unique().tolist()
artist_info_df = get_artist_info_df(artist_id_list)

## 3.5. Combine streaming history, track info, audio features, and artist info

In [ ]:
# Write a function to combine all dataframes, including track_reference_df, audio_feature_df, and artist_info_df
def combine_dataframes():
    df = streaming_hist_df.merge(track_reference_df, how='left', on=['trackName','artistName'])
    df = df.merge(audio_feature_df, how='left', on=['trackId'])
    df = df.merge(artist_info_df, how='left', on=['artistId'])
    df = df.drop_duplicates().reset_index(drop=True)
    return df

In [ ]:
# Apply the function to combine all dataframes
spotify_df = combine_dataframes()

## 3.6. Remove missing entries

In [ ]:
# Write a function to calculate the percentage of null values in the dataframe
def check_missing_values(df):
    for index, value in df.items():
        count_null = value.isnull()
        percentage_null = count_null.sum() * 100.0 / np.shape(df)[0]
        print("Percentage of null values in column %s = %.2f%%" % (index, percentage_null))

In [ ]:
# Apply the function to calculate the percentage of null values in the dataframe
check_missing_values(spotify_df)

In [ ]:
# Write a funciton to remove rows with missing values
def clean_spotify_df(df):
    df = df[df['trackName'].notnull()]
    df = df[df['trackId'].notnull()]
    df = df[df['danceability'].notnull()]
    df = df[df['artistGenre'].notnull()]
    df = df.drop(columns={'username','episode_name','episode_show_name','spotify_episode_uri','skipped'})
    df = df.reset_index(drop=True)
    return df

In [ ]:
# Apply the function to remove rows with missing values
spotify_df = clean_spotify_df(spotify_df)

# Check again if there are still some missing values in the dataframe
check_missing_values(spotify_df)

##  3.7. Handling datetime

In the original data, we only have `ts`, representing the timestamp of when the stream ended in UTC format. Therefore, we will transform the data and add several fields to support our analysis.

In [ ]:
# Write a function to change ts from UTC to local time
def convert_to_local_time(df, col):
    # Create an empty list
    clean_ts_local = []
    
    # Declare some date threshold
    start_date = pd.Timestamp(pd.to_datetime('2021-09-04', format='%Y-%m-%d %H:%M:%S')).tz_localize('UTC')
    end_date = pd.Timestamp(pd.to_datetime('2022-09-07', format='%Y-%m-%d %H:%M:%S')).tz_localize('UTC')
    bst_date = pd.Timestamp(pd.to_datetime('2021-10-31', format='%Y-%m-%d %H:%M:%S')).tz_localize('UTC')
    
    # Convert to local time
    for index, value in col.items():
        datetime = pd.to_datetime(value, format='%Y-%m-%d %H:%M:%S')
        if datetime < start_date or datetime > end_date: # When I lived in Jakarta
            clean_ts_local.append(datetime + timedelta(hours=7))
        elif datetime >= start_date and datetime < bst_date: # When I lived in the UK BST
            clean_ts_local.append(datetime + timedelta(hours=1))
        else:
            clean_ts_local.append(datetime) # When I lived in the UK GMT
        
    # Append the result into a new column in the dataframe
    df['ts_local'] = clean_ts_local
    
    return df

In [ ]:
# Write a function to transform milisecons into seconds and minutes
def transform_duration(df):
    # Fill NA with 0
    df[['ms_played', 'duration_ms']] = df[['ms_played','duration_ms']].fillna(value=0)
    
    # Create a column of duration in second
    df['sec_played'] = df['ms_played']/1000
    df['duration_sec'] = df['duration_ms']/1000
    
    # Create a column of duration in second
    df['min_played'] = df['ms_played']/1000/60
    df['duration_min'] = df['duration_ms']/1000.60
    
    return df

In [ ]:
# Write a function to extract year, month, date, hour from the timestamp ts_local and albumReleaseDate
def extract_year_month_date_hour(df):
    # Step 1: Transform ts_local
    # Create empty lists
    year_ts_local = []
    month_ts_local = []
    date_ts_local = []
    hour_ts_local = []
    
    # Extract year, year-month, year-month-date, and hour from the timestamp
    for index, value in df['ts_local'].items():
        year_ts_local.append(value.strftime('%Y'))
        month_ts_local.append(value.strftime('%Y-%m'))
        date_ts_local.append(value.strftime('%Y-%m-%d'))
        hour_ts_local.append(value.strftime('%H'))
        
    # Append the results into new columns in the dataframe
    df['year_ts_local'] = year_ts_local
    df['month_ts_local'] = month_ts_local 
    df['date_ts_local'] = date_ts_local
    df['hour_ts_local'] = hour_ts_local

    # Step 2: Transform albumReleaseDate
    # Create an empty list
    release_year = []
    
    # Extract albumReleaseYear
    for index, value in df['albumReleaseDate'].items():
        try:
            datetime = pd.to_datetime(value, format='%Y-%m-%d')
            release_year.append(datetime.strftime('%Y'))
        except Exception:
            release_year.append('Unidentified')

    # Append the result into a new column in the dataframe
    df['albumReleaseYear'] = release_year
    
    return df

In [ ]:
# Write a function to group the timestamps into its respective time category
def append_time_category(df):
    # Transform hour_ts_local into a list
    hour_ts_local_list = df['hour_ts_local'].astype('int').to_list()
    
    # Create an empty list
    time_cat_list = []
    
    # Write conditions to group the timestamps
    for value in hour_ts_local_list:
        if value >= 0 and value <= 5:
            time_cat_list.append('00.00 - 05.59')
        elif value >=6 and value <= 11:
            time_cat_list.append('06.00 - 11.59')
        elif value >= 12 and value <= 17:
            time_cat_list.append('12.00 - 17.59')
        elif value >= 18 and value <= 23:
            time_cat_list.append('18.00 - 23.59')
        else:
            time_cat_list.append('Unidentified')
    
    # Append the result into a new column in the dataframe
    df['ts_category'] = time_cat_list
    
    return df

In [ ]:
# Change endTime from UTC to local time
spotify_df = convert_to_local_time(spotify_df, spotify_df['ts'])

# Transform duration
spotify_df = transform_duration(spotify_df)

# Extract year, month, date, hour
spotify_df = extract_year_month_date_hour(spotify_df)

# Append time category
spotify_df = append_time_category(spotify_df)

# 4. Exploratory Data Analysis

## 4.1. Time spent on Spotify

In [ ]:
# Write a function to compute daily time spent on Spotify
def daily_time_spent(df):
    daily_plays = df.groupby(['year_ts_local','month_ts_local','date_ts_local'], as_index=False).agg({'sec_played':'sum'})
    daily_plays['min_played'] = round(daily_plays['sec_played']/60,2)
    daily_plays['hour_played'] = round(daily_plays['min_played']/60,2)
    return daily_plays

In [ ]:
# Write a function to compute monthly time spent on Spotify
def monthly_time_spent(df):
    monthly_plays = df.groupby(['year_ts_local','month_ts_local'], as_index=False).agg({'sec_played':'sum'})
    monthly_plays['min_played'] = round(monthly_plays['sec_played']/60,2)
    monthly_plays['hour_played'] = round(monthly_plays['min_played']/60,2)
    return monthly_plays

In [ ]:
# Write a function to compute yearly time spent on Spotify
def yearly_time_spent(df):
    yearly_plays = df.groupby('year_ts_local', as_index=False).agg({'sec_played':'sum'})
    yearly_plays['min_played'] = round(yearly_plays['sec_played']/60,2)
    yearly_plays['hour_played'] = round(yearly_plays['sec_played']/60,2)
    return yearly_plays

In [ ]:
# Compute daily time spent on Spotify
daily_time_spent(spotify_df)

In [ ]:
# Compute monthly time spent on Spotify
monthly_time_spent(spotify_df)

In [ ]:
# Compute yearly time spent on Spotify
yearly_time_spent(spotify_df)

## 4.2. Top tracks by year

In [ ]:
# Write a function to get top tracks by total minutes played
# This following function retrieves the top 10 tracks of each year by total minutes played
def top_tracks_by_year(df):
    result_df = df.groupby(['year_ts_local','trackName','artistName','artistGenre','artistPopularity'], as_index=False).agg({'sec_played':'sum'})
    result_df['min_played'] = round(result_df['sec_played']/60,2)
    result_df = result_df.sort_values(['sec_played'], ascending=False).groupby('year_ts_local').head(10)
    result_df = result_df.sort_values(['year_ts_local'],ascending=True)
    return result_df

In [ ]:
# Apply the function to get the top tracks
top_tracks_by_year(spotify_df)

## 4.3. Top artists by year

In [ ]:
# Write a function to get top artists by total minutes played
# This following function retrieves the top 10 artists of each year by total minutes played
def top_artists_by_year(df):
    result_df = df.groupby(['year_ts_local','artistId','artistName','artistGenre','artistPopularity'], as_index=False).agg({'sec_played':'sum','trackId':pd.Series.nunique})
    result_df['min_played'] = round(result_df['sec_played']/60,2)    
    result_df = result_df.rename(columns={'trackId':'unique_tracks'})
    result_df = result_df.sort_values(['min_played'], ascending=False).groupby('year_ts_local').head(10)
    result_df = result_df.sort_values(['year_ts_local','min_played'],ascending=False)
    result_df = result_df.reset_index(drop=True)
    return result_df

In [ ]:
# Apply the function to get the top artists
top_artists_by_year(spotify_df)

## 4.4. The days I discovered my favorite artists

In [ ]:
# The day I discovered my top artists
def the_day_I_discovered_my_favorite_artists(df):
    # Write a function to get top artists by total minutes played
    # This following function retrieves the top 10 artists of each year by total minutes played
    result_df = df.groupby(['year_ts_local','artistId','artistName','artistGenre','artistPopularity'], as_index=False).agg({'sec_played':'sum','trackId':pd.Series.nunique})
    result_df['min_played'] = round(result_df['sec_played']/60,2)    
    result_df = result_df.rename(columns={'trackId':'unique_tracks'})
    result_df = result_df.sort_values(['min_played'], ascending=False).groupby('year_ts_local').head(10)
    result_df = result_df.sort_values(['year_ts_local','min_played'],ascending=False)
    result_df = result_df.reset_index(drop=True)
    
    # Write a function to get the first date of listening
    first_listen_df = df.groupby(['artistId'], as_index=False).agg({'date_ts_local':'min'})
    first_listen_df = first_listen_df.rename(columns={'date_ts_local':'first_listen_date'})
    
    # Combine data
    result_df = result_df.merge(first_listen_df, how='left', on=['artistId'])
    return result_df

In [ ]:
the_day_I_discovered_my_favorite_artists_df = the_day_I_discovered_my_favorite_artists(spotify_df)
the_day_I_discovered_my_favorite_artists_df[the_day_I_discovered_my_favorite_artists_df['year_ts_local']=='2022']

## 4.5. Top genre by year

In [ ]:
# Write a function to get top genres by total minutes played
# This following function retrieves the top 10 genres of each year by total minutes played
def top_genre_by_year(df):
    df = df[df['artistGenre']!='Unidentified']
    result_df = df.groupby(['year_ts_local','artistGenre'], as_index=False).agg({'sec_played':'sum'})
    result_df['min_played'] = round(result_df['sec_played']/60,2)
    result_df = result_df.sort_values(['min_played'], ascending=False).groupby('year_ts_local').head(10)
    result_df = result_df.sort_values(['year_ts_local','min_played'],ascending=False)
    result_df = result_df.reset_index(drop=True)
    return result_df

In [ ]:
# Apply the function to get the top genres
top_genre_by_year(spotify_df)

## 4.6. Most repeated songs

In [ ]:
# Write a function to get the most repeated song of each year
def most_repeated_song_by_year(df):
    # Sort the dataframe by the timestamp
    df = df.sort_values('ts_local',ascending=True)
    
    # Retrieve the previously played track (Equivalent to LAG function in SQL)
    df['previous_trackId'] = df['trackId'].shift(1)
    
    # Write a condition to flag whether the track being played is repeated or not
    df['is_repeated'] = np.where(df['trackId']==df['previous_trackId'], 1, 0)
    
    # Compute the number of repeats with pd.Series.cumsum
    df['count_repeated'] = df.groupby(['year_ts_local','date_ts_local','trackId','trackName','artistName','artistGenre'])['is_repeated'].transform(pd.Series.cumsum)
    df = df.groupby(['year_ts_local','date_ts_local','trackId','trackName','artistName','artistGenre'],as_index=False).agg({'count_repeated':'max'}).sort_values('count_repeated',ascending=False)
    
    # Filter only the most repeated song for each year
    df = df.loc[df.groupby('year_ts_local')['count_repeated'].idxmax()][['year_ts_local','trackId','trackName','artistName','artistGenre','count_repeated']].reset_index(drop=True)

    return df

In [ ]:
# Apply the function to get the most repeated song by year
most_repeated_song_by_year(spotify_df)

In [ ]:
# Write a function to get the most repeated songs by month (2022 only)
def most_repeated_song_by_month(df):
    # Filter to 2022 only and sort the dataframe by the timestamp
    df = df[df['year_ts_local']=='2022'].reset_index(drop=True)
    df = df.sort_values('ts_local',ascending=True).reset_index(drop=True)
    
    # Retrieve the previously played track (Equivalent to LAG function in SQL)    
    df['previous_trackId'] = df['trackId'].shift(1)
    
    # Write a condition to flag whether the track being played is repeated or not    
    df['is_repeated'] = np.where(df['trackId']==df['previous_trackId'], 1, 0)
    
    # Compute the number of repeats with pd.Series.cumsum
    df['count_repeated'] = df.groupby(['month_ts_local','date_ts_local','trackId','trackName','artistName'])['is_repeated'].transform(pd.Series.cumsum)
    df = df.groupby(['month_ts_local','date_ts_local','trackId','trackName','artistName'],as_index=False).agg({'count_repeated':'max'}).sort_values('count_repeated',ascending=False)
    
    # Filter only the most repeated song for each month in 2022
    df = df.loc[df.groupby('month_ts_local')['count_repeated'].idxmax()][['month_ts_local','trackId','trackName','artistName','count_repeated']].reset_index(drop=True)

    return df

In [ ]:
# Apply the function
most_repeated_song_by_month(spotify_df)

## 4.7. Do I listen more to common or unique artists?

In [ ]:
# Write a function to compute total minutes played by artistId as well as identify if an artist is Unique of Common
def artistPopularity_min_played(df):
    # Append a flag to identify whether an artist is unique or common
    artistPopularity_list = df['artistPopularity'].astype('int').to_list()
    artist_flag = []
    for i in list(range(0,len(artistPopularity_list))):
        if artistPopularity_list[i] <= 50:
            artist_flag.append('Unique')
        else:
            artist_flag.append('Common')
    
    df['artistPopularityCat'] = artist_flag
    
    # Compute total minutes played by artistId
    result_df = df.groupby(['artistId', 'artistName','artistGenre','artistPopularity', 'artistPopularityCat','year_ts_local'], as_index=False).agg({'sec_played':'sum','trackId':pd.Series.nunique})
    result_df['min_played'] = round(result_df['sec_played']/60,2)
    
    # Rename columns for intuitiveness
    result_df = result_df.rename(columns={'trackId':'unique_tracks'})
    
    # Remove uninformative entries
    result_df = result_df[(result_df['sec_played']>0) & (result_df['unique_tracks']<=100)].reset_index(drop=True)
    
    # Sort by total minutes played
    result_df = result_df.sort_values(['min_played'],ascending=False)
    
    return result_df

In [ ]:
# Apply the function
unique_vs_common_artist_df = artistPopularity_min_played(spotify_df)

# Aggregate the data
unique_vs_common_artist_df = unique_vs_common_artist_df.groupby(['year_ts_local','artistPopularityCat'], as_index=False).agg({'sec_played':'sum', 'unique_tracks':'sum'})

In [ ]:
# Apply the function
unique_vs_common_artist_raw_df = artistPopularity_min_played(spotify_df)

## 4.8. Top genre by period of the day

In [ ]:
def top_genre_by_time_category(df):
    # Remove tracks with unidentified genre
    df = df[df['artistGenre']!='Unidentified'].reset_index(drop=True)
    
    # Aggregate data
    result_df = df.groupby(['year_ts_local','ts_category','artistGenre'], as_index=False).agg({'sec_played':'sum'})
    result_df['min_played'] = round(result_df['sec_played']/60,2)
    result_df = result_df.sort_values(['min_played'], ascending=False).groupby(['year_ts_local','ts_category']).head(5)
    result_df = result_df.sort_values(['year_ts_local','ts_category'])
    
    return result_df

In [ ]:
# Apply the function to get top genres by period of the day
top_genre_by_time_cat_df = top_genre_by_time_category(spotify_df)

## 4.9. Audio features throughout the day

In [ ]:
def audio_characteristics_throughout_the_day(df):
    # Filter 2022 only
    df = df[df['year_ts_local']=='2022'].reset_index(drop=True)
    
    # Aggregate data
    result_df = df.groupby(['hour_ts_local'], as_index=False).agg({'danceability':'median',
                                                                 'energy':'median',
                                                                 'loudness':'median',
                                                                 'speechiness':'median',
                                                                 'acousticness':'median',
                                                                 'liveness':'median',
                                                                 'instrumentalness':'median',
                                                                 'valence':'median',
                                                                 'tempo':'median'})
    
    
    result_df = pd.melt(result_df, id_vars=['hour_ts_local'], value_vars=['danceability','energy','loudness','speechiness','acousticness','instrumentalness','valence','tempo'])
    
    return result_df

In [ ]:
audio_char_throughout_the_day_df = audio_characteristics_throughout_the_day(spotify_df)

## 4.10. Songs I have a love-hate relationship with

In [ ]:
# Write a function to show tracks that appear in the top 100 but also frequently skipped
def love_hate_songs(df):
    # Filter 2022 only
    df = df[df['year_ts_local']=='2022'].reset_index(drop=True)
    
    # Declare inputs
    sec_played_list = df['sec_played'].to_list()
    duration_sec = df['duration_sec'].to_list()
    reason_end_list = df['reason_end'].to_list()
    df_length = len(df)
    pct_played_list = []
    is_skipped_list = []
    
    for i in list(range(0,df_length)):
        # Compute percentage of track played
        pct_played = round(sec_played_list[i]/duration_sec[i],2)
        pct_played_list.append(pct_played)
        
        # Add a flag to identify if a song is skipped
        if reason_end_list[i] != 'trackdone':
            is_skipped_list.append(1)
        else:
            is_skipped_list.append(0)   
        
    # Append the result to the dataframe
    df['pct_played'] = pct_played_list
    df['is_skipped'] = is_skipped_list
    
    # Get the number of skips by trackId
    skipped_df = df.groupby(['trackId','trackName'], as_index=False).agg({'is_skipped':'sum'}).rename(columns={'is_skipped':'count_skipped'})
    
    # Get top 100 tracks in 2022 by number of played
    top_100_tracks_df = df.groupby(['trackId','trackName'], as_index=False).size().rename(columns={'size':'count_played'})
    top_100_tracks_df = top_100_tracks_df.sort_values(['count_played'], ascending=False).head(100)
    
    # Get love-hate songs (Appear in top 100 tracks but also frequently skipped)
    love_hate_df = top_100_tracks_df.merge(skipped_df, how='left', on=['trackId','trackName'])
    love_hate_df['pct_skipped'] = round(love_hate_df['count_skipped']/love_hate_df['count_played']*100,2)
    love_hate_df = love_hate_df.sort_values('pct_skipped',ascending=False).head(10)
    
    return love_hate_df

In [ ]:
# Apply the function
love_hate_songs_df = love_hate_songs(spotify_df)

## 4.11. Songs I love at the first listen

In [ ]:
def love_at_the_first_listen(df): 
    # first date of listening
    first_listen_df = df.groupby(['trackId'], as_index=False).agg({'date_ts_local':'min'})
    first_listen_df = first_listen_df.rename(columns={'date_ts_local':'first_listen_date'})
    
    # number of listening by date
    daily_listen_df = df.groupby(['trackId','trackName','artistName','date_ts_local','year_ts_local'], as_index=False).size()
    daily_listen_df = daily_listen_df.rename(columns={'size':'num_listen'})
    
    # count repeat by date
    count_repeat_df = df.sort_values('ts_local',ascending=True)
    count_repeat_df['previous_trackId'] = count_repeat_df['trackId'].shift(1)
    count_repeat_df['is_repeated'] = np.where(count_repeat_df['trackId']==count_repeat_df['previous_trackId'], 1, 0)
    count_repeat_df['count_repeated'] = count_repeat_df.groupby(['trackId','trackName','artistName','date_ts_local','year_ts_local'])['is_repeated'].transform(pd.Series.cumsum)
    count_repeat_df = count_repeat_df.groupby(['trackId','trackName','artistName','date_ts_local','year_ts_local'],as_index=False).agg({'count_repeated':'max'})
    
    # combine dataframes
    result_df = count_repeat_df.merge(first_listen_df, how='left', on='trackId')
    result_df = result_df[result_df['first_listen_date']==result_df['date_ts_local']]
    result_df = result_df.sort_values('count_repeated', ascending=False).head(50)

    # filter 2022
    result_df = result_df[result_df['year_ts_local']=='2022'].reset_index(drop=True)
    
    return result_df

In [ ]:
love_at_the_first_listen(spotify_df)

## 4.12. Songs I skipped the first time but realized it was bomb much later

In [ ]:
def skipped_first_repeated_later(df):
    # Count percentage played (ms_played/duration_ms)
    pct_played_df = df[['trackId','ts_local','ms_played','duration_ms']].reset_index(drop=True)
    pct_played_df['pct_played'] = round(pct_played_df['ms_played']/pct_played_df['duration_ms']*100,2)
    
    # Extract first listen timestamp
    first_listen_df = df.groupby(['trackId'], as_index=False).agg({'ts_local':'min'})
    first_listen_df = first_listen_df.merge(pct_played_df, how='inner', on=['trackId','ts_local'])
    
    # Extract songs that I skipped at the first listen
    reason_end_list = ['endplay','fwdbtn','backbtn']
    df_filtered = df[['trackId','trackName','artistName','ts_local','reason_end']]
    skipped_first_listen_df = first_listen_df.merge(df_filtered, how='inner', on=['trackId','ts_local'])
    skipped_first_listen_df = skipped_first_listen_df[skipped_first_listen_df['reason_end'].isin(reason_end_list)][['trackId','trackName','artistName','ts_local','reason_end','ms_played','duration_ms','pct_played']]
    skipped_first_listen_df = skipped_first_listen_df.rename(columns={'ts_local':'first_listen_ts_local'})
    
    # Extract date from first_listen_ts_local
    first_listen_date_ts_local = []
    for index, value in skipped_first_listen_df['first_listen_ts_local'].items():
         first_listen_date_ts_local.append(value.strftime('%Y-%m-%d'))
    skipped_first_listen_df['first_listen_date_ts_local'] =  first_listen_date_ts_local
       
    # Count repeat, grouped by date
    count_repeat_df = df.sort_values('ts_local',ascending=True)
    count_repeat_df['previous_trackId'] = count_repeat_df['trackId'].shift(1)
    count_repeat_df['is_repeated'] = np.where(count_repeat_df['trackId']==count_repeat_df['previous_trackId'], 1, 0)
    count_repeat_df['count_repeated'] = count_repeat_df.groupby(['trackId','trackName','artistName','date_ts_local','year_ts_local'])['is_repeated'].transform(pd.Series.cumsum)
    count_repeat_df = count_repeat_df.groupby(['trackId','trackName','artistName','date_ts_local','year_ts_local'],as_index=False).agg({'count_repeated':'max'})
    count_repeat_df['rank'] = count_repeat_df.groupby(['trackId','trackName','artistName'])['count_repeated'].rank(ascending=False)
    count_repeat_df = count_repeat_df[count_repeat_df['rank']==1].reset_index(drop=True)
    
    # Combine dataframe
    result_df = count_repeat_df.merge(skipped_first_listen_df, how='inner', on=['trackId','trackName','artistName'])
    result_df = result_df.sort_values('count_repeated',ascending=False)
    
    # Rearrange dataframe
    result_df = result_df[['trackId','trackName','artistName','first_listen_date_ts_local','date_ts_local','year_ts_local','count_repeated','reason_end','ms_played','duration_ms','pct_played']]
    result_df = result_df[result_df['pct_played']<100]
    
    # Filter top 10
    result_df = result_df[result_df['year_ts_local']=='2022'].head(10).reset_index(drop=True)
    
    return result_df

In [ ]:
skipped_first_repeated_later(spotify_df)

## 4.13. Do I tend to be more explorative over the years?

In [ ]:
def pct_new_artists_by_year(df):    
    # Extract first listen timestamp
    first_listen_df = df.groupby(['artistId'], as_index=False).agg({'ts_local':'min'})
    first_listen_df = first_listen_df.rename(columns={'ts_local':'first_listen_ts_local'})
    
    # Extract year from first_listen_ts_local
    first_listen_year_ts_local = []
    for index, value in first_listen_df['first_listen_ts_local'].items():
         first_listen_year_ts_local.append(value.strftime('%Y'))
    first_listen_df['first_listen_year_ts_local'] =  first_listen_year_ts_local
    
    # Combine dataframe
    result_df = df.merge(first_listen_df, how='left', on=['artistId']).reset_index(drop=True)
    
    # Append new artist flag
    first_listen_year_list = result_df['first_listen_year_ts_local'].to_list()
    year_played_list = result_df['year_ts_local'].to_list()
    
    new_artist_flag = []
    for i in list(range(0, len(result_df))):
        if first_listen_year_list[i] == year_played_list[i]:
            new_artist_flag.append('New artist')
        else:
            new_artist_flag.append('Familiar artist')
            
    result_df['new_artist_flag'] = new_artist_flag
    
    # Count total unique artists by year
    unique_artist_df = result_df.groupby(['year_ts_local'],as_index=False).agg({'artistId':pd.Series.nunique})
    unique_artist_df = unique_artist_df.rename(columns={'artistId':'unique_artists'})
    
    # Count new artists by year
    new_artist_df = result_df.groupby(['year_ts_local','new_artist_flag'], as_index=False).agg({'artistId':pd.Series.nunique})
    #new_artist_df = new_artist_df.pivot(index='year_ts_local', columns='new_artist_flag', values='artistId').reset_index(drop=False)
    #new_artist_df = new_artist_df[['year_ts_local','True']].rename(columns={'True':'unique_new_artists'})
    
    # Combine dataframes
    #final_df = unique_artist_df.merge(new_artist_df, how='left', on=['year_ts_local'])
    #final_df['pct_new_artists'] = round(final_df['unique_new_artists']/final_df['unique_artists']*100,2)
    
    return new_artist_df

In [ ]:
pct_new_artists_by_year(spotify_df)

In [ ]:
def pct_new_tracks_by_year(df):    
    # Extract first listen timestamp
    first_listen_df = df.groupby(['trackId'], as_index=False).agg({'ts_local':'min'})
    first_listen_df = first_listen_df.rename(columns={'ts_local':'first_listen_ts_local'})
    
    # Extract year from first_listen_ts_local
    first_listen_year_ts_local = []
    for index, value in first_listen_df['first_listen_ts_local'].items():
         first_listen_year_ts_local.append(value.strftime('%Y'))
    first_listen_df['first_listen_year_ts_local'] =  first_listen_year_ts_local
    
    # Combine dataframe
    result_df = df.merge(first_listen_df, how='left', on=['trackId']).reset_index(drop=True)
    
    # Append new artist flag
    first_listen_year_list = result_df['first_listen_year_ts_local'].to_list()
    year_played_list = result_df['year_ts_local'].to_list()
    
    new_track_flag = []
    for i in list(range(0, len(result_df))):
        if first_listen_year_list[i] == year_played_list[i]:
            new_track_flag.append('New track')
        else:
            new_track_flag.append('Familiar track')
            
    result_df['new_track_flag'] = new_track_flag
    
    # Count total unique tracks by year
    unique_track_df = result_df.groupby(['year_ts_local'],as_index=False).agg({'trackId':pd.Series.nunique})
    unique_track_df = unique_track_df.rename(columns={'trackId':'unique_tracks'})
    
    # Count new tracks by year
    new_track_df = result_df.groupby(['year_ts_local','new_track_flag'], as_index=False).agg({'trackId':pd.Series.nunique})
    #new_track_df = new_track_df.pivot(index='year_ts_local', columns='new_track_flag', values='trackId').reset_index(drop=False)
    #new_track_df = new_track_df[['year_ts_local','True']].rename(columns={'True':'unique_new_tracks'})
    
    # Combine dataframes
    #final_df = unique_track_df.merge(new_track_df, how='left', on=['year_ts_local'])
    #final_df['pct_new_tracks'] = round(final_df['unique_new_tracks']/final_df['unique_tracks']*100,2)
    
    return new_track_df

In [ ]:
pct_new_tracks_by_year(spotify_df)

## 2.14. Songs I no longer love as much as I did

In [ ]:
def songs_I_dont_love_anymore(df):
    # Filter year
    df = df[df['year_ts_local'].isin(['2021','2022'])].reset_index(drop=True)
    
    # Total ms played
    total_ms_played_df = df.groupby(['year_ts_local'], as_index=False).agg({'ms_played':'sum'})
    total_ms_played_2021 = total_ms_played_df[total_ms_played_df['year_ts_local']=='2021']['ms_played'][0]
    total_ms_played_2022 = total_ms_played_df[total_ms_played_df['year_ts_local']=='2022']['ms_played'][1]
    overall_pct_growth = round(((total_ms_played_2022/total_ms_played_2021)-1)*100,2)
    
    result_df = df.groupby(['trackId','year_ts_local'], as_index=False).agg({'ms_played':'sum'})
    result_df = result_df.pivot(index='trackId', columns='year_ts_local', values='ms_played').reset_index(drop=False)
    result_df = result_df[(result_df['2021'].notnull()) & (result_df['2022'].notnull())]
    result_df = result_df.sort_values('2021', ascending=False).head(50).reset_index(drop=True)
    result_df['pct_growth'] = round(((result_df['2022']/result_df['2021'])-1)*100,2)
    
    # Filter
    result_df = result_df[result_df['pct_growth']<overall_pct_growth]
    
    # Append trackName and artistName
    result_df = result_df.merge(track_reference_df[['trackId','trackName','artistName']], how='left', on=['trackId'])
    result_df['rank'] = result_df.groupby(['trackId'])['pct_growth'].rank(ascending=False)
    result_df = result_df[result_df['rank']==1].drop(columns={'rank'})
    result_df = result_df.sort_values(['pct_growth'], ascending=True).head(10)
    
    result_df = pd.melt(result_df, id_vars=['trackId','trackName','artistName','pct_growth'], value_vars=['2021','2022'])
    result_df = result_df.rename(columns={'variable':'year', 'value':'ms_played'})
    
    return result_df

In [ ]:
songs_I_dont_love_anymore(spotify_df)

## 2.15. Songs from the past that I am still listening today

In [ ]:
# Write a function to show tracks that appear in the top 100 and released more than 30 years ago
def songs_from_the_past(df):
    # Filter 2022 only
    df = df[df['year_ts_local']=='2022'].reset_index(drop=True)
    
    # Get top 100 tracks in 2022 by number of played
    top_100_tracks_df = df.groupby(['trackId','trackName'], as_index=False).size().rename(columns={'size':'count_played'})
    top_100_tracks_df = top_100_tracks_df.sort_values(['count_played'], ascending=False).head(100)
    
    # Get the old souls in me
    old_songs_df = df[df['albumReleaseYear']<='1992'][['trackId','trackName','artistName','albumReleaseYear']].drop_duplicates().reset_index(drop=True)
    
    old_songs_df = old_songs_df.merge(top_100_tracks_df, how='inner', on=['trackId','trackName'])

    return old_songs_df

In [ ]:
songs_from_the_past(spotify_df)